In [2]:
import pandas as pd
import os
import random

os.getcwd()

'/home/nn/pattern-transformer'

In [3]:
dfs = []
for root,dirs,files in os.walk('./data'):
    for i in files:
        df = pd.read_csv(os.path.join(root, i))
        dfs.append(df)

In [4]:
print('目前出现logic inconsistency的query有', len(dfs))

目前出现logic inconsistency的query有 13559


In [7]:
print('随便取一个查看')
random.choice(dfs)

随便取一个查看


,"['A', 'TPo']","['1', 'A']"
0,1.0,NaN
1,NaN,1.0


现在想要大致对这些执行结果分个类，数据特征类似的大概率是同一种bug

In [8]:
# 先按数量是否相等来划分这类
sum_eqs = []
sum_neqs = []
for df in dfs[5:]:
    sum_row = df.sum(axis=1)
    if sum_row[0] != sum_row[1]:
        sum_neqs.append(df)
    else:
        sum_eqs.append(df)

In [9]:
print("两个query的结果数量相同的有:", len(sum_eqs))
print("两个query的结果数量不相同的有:", len(sum_neqs))

两个query的结果数量相同的有: 12648
两个query的结果数量不相同的有: 906


In [10]:
random.choice(sum_eqs)

,"[-21931217, -940276028, 'egAtcmP']","[-21931217, 1398361557, 'egAtcmP']"
0,5022,4978
1,5004,4996


In [11]:
random.choice(sum_neqs)

,"[2036216611, '5vxfC9r', None, 'T']","[-1442164853, '5vxfC9r', None, 'T']","['5vxfC9r', 727428285, None, 'T']","[208116405, '5vxfC9r', None, 'T']","['5vxfC9r', 817958452, None, 'T']","[1307760644, '5vxfC9r', None, 'T']","[574705761, '5vxfC9r', None, 'T']","['5vxfC9r', 946496522, None, 'T']"
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# 再通过执行结果是否有缺失值来划分。有缺失值意味着两个query里的response key不一致，没有则意味着key是一致的，只是数量不一致
def hasNa(dfs):
    sum_eqs_hasna = []
    sum_eqs_nona = []

    for df in dfs:
        if df.isna().any().any():
            sum_eqs_hasna.append(df)
        else:
            sum_eqs_nona.append(df)
    return sum_eqs_hasna, sum_eqs_nona


In [13]:
sum_eqs_hasna, sum_eqs_nona  = hasNa(sum_eqs)
sum_neqs_hasna, sum_neqs_nona  = hasNa(sum_neqs)

In [14]:
random.choice(sum_eqs_hasna)

,"['M', True, True]","[True, True, 'z']"
0,2.0,NaN
1,NaN,2.0


In [15]:
random.choice(sum_eqs_nona)

,"[-1416545915, '8M902v']","[-1267638192, '8M902v']","[-532357484, '8M902v']","[-605306270, '8M902v']","[-941744231, '8M902v']","[1694616852, '8M902v']","[-127977559, '8M902v']","['8M902v', None]"
0,199,91,115,115,46,66,44,66
1,203,90,112,112,45,67,45,68


In [16]:
random.choice(sum_neqs_hasna)

,"[-1646819731, -652029211, False]","[-1646819731, 1679683596, False]"
0,4.0,NaN
1,NaN,2.0


In [17]:
random.choice(sum_neqs_nona)

,"[None, True]"
0,12
1,6


In [18]:
print(len(sum_eqs_hasna))
print(len(sum_eqs_nona))
print(len(sum_neqs_hasna))
print(len(sum_neqs_nona))

9532
3116
590
316


从这个结果来看，数量相等，且有na值的占大头，其余的较少。后续要做的是手动reduce一些bug出来，看看这个划分对于bug的查找有没有帮助。如果正好对应的话，这划分出来的4种可能能对应4种bug。

如果结果对不上，就看看有没有其他的划分数据的办法，将bug聚类一下